In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\mktmi\AppData\Local\Temp\ipykernel_14144\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [7]:
# Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("C:\\Users\\mktmi\\Documents\\ironhack\\AI_Engineering\\Work\\Week4\\lab-natural-language-processing-1\\data\\kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
data = data.head(1000)

# Print the shape of the data
print(data.shape)

# Fill missing values with an empty string
data.fillna("", inplace=True)


(1000, 2)


### Let's divide the training and test set into two partitions

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("C:/Users/mktmi/Documents/ironhack/AI_Engineering/Work/Week4/lab-natural-language-processing-1/data/kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
data = data.head(1000)

# Fill missing values with an empty string
data.fillna("", inplace=True)

# Assuming 'label' is the target column and all others are features
X = data.drop(columns=['label'])  # Features
y = data['label']  # Target variable

# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes to verify the split
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)



Training set shape: (800, 1) (800,)
Testing set shape: (200, 1) (200,)


## Data Preprocessing

In [9]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [10]:
import re
from bs4 import BeautifulSoup

def clean_html(html_content):
    # Step 1: Remove inline JavaScript/CSS
    # Using regex to remove content within <script> and <style> tags
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    # Removing comments using regex
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)
    
    # Step 3: Remove remaining HTML tags
    # Using BeautifulSoup to strip the remaining HTML tags
    soup = BeautifulSoup(html_content, 'html.parser')
    cleaned_text = soup.get_text()
    
    # Optional: You can further clean the text, e.g., removing extra whitespace or special characters
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Replace multiple spaces with a single space
    
    return cleaned_text.strip()

# Example usage on some HTML content
html_content = """
<html>
<head><style>body {font-size: 12px;}</style><script>alert('This is JS');</script></head>
<body>
    <!-- This is a comment -->
    <h1>This is a header</h1>
    <p>This is a <strong>paragraph</strong> with <a href="#">a link</a>.</p>
</body>
</html>
"""

cleaned_text = clean_html(html_content)
print("Cleaned text:", cleaned_text)


Cleaned text: This is a header This is a paragraph with a link.


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [12]:
import re
from bs4 import BeautifulSoup

def clean_html(html_content):
    # Step 1: Remove inline JavaScript/CSS
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)
    
    # Step 3: Remove remaining HTML tags
    soup = BeautifulSoup(html_content, 'html.parser')
    cleaned_text = soup.get_text()

    # Step 4: Remove special characters (keeping only letters and spaces)
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', cleaned_text)
    
    # Step 5: Remove numbers
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    
    # Step 6: Remove all single characters
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    
    # Step 7: Remove single characters from the start
    cleaned_text = re.sub(r'\b\w{1}\s', '', cleaned_text)
    
    # Step 8: Substitute multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Step 9: Remove prefixed 'b' (if present in text encoding issues)
    cleaned_text = re.sub(r'^b\s+', '', cleaned_text)
    
    # Step 10: Convert to lowercase
    cleaned_text = cleaned_text.lower()

    # Strip leading/trailing spaces
    return cleaned_text.strip()

# Example usage on some HTML content
html_content = """
<html>
<head><style>body {font-size: 12px;}</style><script>alert('This is JS');</script></head>
<body>
    <!-- This is a comment -->
    <h1>This is a header 123</h1>
    <p>This is a b <strong>paragraph</strong> with 56 <a href="#">a link</a> & special characters!!!.</p>
</body>
</html>
"""

cleaned_text = clean_html(html_content)
print("Cleaned text:", cleaned_text)



Cleaned text: this is header this is paragraph with link special characters


## Now let's work on removing stopwords
Remove the stopwords.

In [14]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk

# Ensure you have downloaded the stopwords dataset from NLTK
nltk.download('stopwords')

def clean_html(html_content):
    # Step 1: Remove inline JavaScript/CSS
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)
    
    # Step 3: Remove remaining HTML tags
    soup = BeautifulSoup(html_content, 'html.parser')
    cleaned_text = soup.get_text()

    # Step 4: Remove special characters (keeping only letters and spaces)
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', cleaned_text)
    
    # Step 5: Remove numbers
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    
    # Step 6: Remove all single characters
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    
    # Step 7: Remove single characters from the start
    cleaned_text = re.sub(r'\b\w{1}\s', '', cleaned_text)
    
    # Step 8: Substitute multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Step 9: Remove prefixed 'b'
    cleaned_text = re.sub(r'^b\s+', '', cleaned_text)
    
    # Step 10: Convert to lowercase
    cleaned_text = cleaned_text.lower()

    # Step 11: Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = cleaned_text.split()
    filtered_words = [word for word in words if word not in stop_words]
    
    # Join the remaining words back into a single string
    cleaned_text = ' '.join(filtered_words)

    # Strip leading/trailing spaces and return cleaned text
    return cleaned_text.strip()

# Example usage on some HTML content
html_content = """
<html>
<head><style>body {font-size: 12px;}</style><script>alert('This is JS');</script></head>
<body>
    <!-- This is a comment -->
    <h1>This is a header 123</h1>
    <p>This is a b <strong>paragraph</strong> with 56 <a href="#">a link</a> & special characters!!!.</p>
</body>
</html>
"""

cleaned_text = clean_html(html_content)
print("Cleaned text:", cleaned_text)


Cleaned text: header paragraph link special characters


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mktmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [15]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def clean_html(html_content):
    # Step 1: Remove inline JavaScript/CSS
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
    
    # Step 2: Remove HTML comments
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)
    
    # Step 3: Remove remaining HTML tags
    soup = BeautifulSoup(html_content, 'html.parser')
    cleaned_text = soup.get_text()

    # Step 4: Remove special characters (keeping only letters and spaces)
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', cleaned_text)
    
    # Step 5: Remove numbers
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    
    # Step 6: Remove all single characters
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    
    # Step 7: Remove single characters from the start
    cleaned_text = re.sub(r'\b\w{1}\s', '', cleaned_text)
    
    # Step 8: Substitute multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Step 9: Remove prefixed 'b'
    cleaned_text = re.sub(r'^b\s+', '', cleaned_text)
    
    # Step 10: Convert to lowercase
    cleaned_text = cleaned_text.lower()

    # Step 11: Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = cleaned_text.split()
    filtered_words = [word for word in words if word not in stop_words]

    # Step 12: Apply Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    # Join the lemmatized words back into a single string
    cleaned_text = ' '.join(lemmatized_words)

    # Strip leading/trailing spaces and return cleaned text
    return cleaned_text.strip()

# Example usage on some HTML content
html_content = """
<html>
<head><style>body {font-size: 12px;}</style><script>alert('This is JS');</script></head>
<body>
    <!-- This is a comment -->
    <h1>Dogs are running faster than cats!</h1>
    <p>This is a b <strong>paragraph</strong> with 56 <a href="#">a link</a> & special characters!!!.</p>
</body>
</html>
"""

cleaned_text = clean_html(html_content)
print("Cleaned text:", cleaned_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mktmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\mktmi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\mktmi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Cleaned text: dog running faster cat paragraph link special character


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [28]:
import re
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Define the clean_html function
def clean_html(html_content):
    # Remove inline JavaScript/CSS
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
    
    # Remove HTML comments
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    soup = BeautifulSoup(html_content, 'html.parser')
    cleaned_text = soup.get_text()

    # Remove special characters (keeping only letters and spaces)
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', cleaned_text)
    
    # Remove numbers
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    
    # Remove single characters and substitute multiple spaces with a single space
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Convert to lowercase
    return cleaned_text.lower().strip()

# Load the dataset
data = pd.read_csv("C:/Users/mktmi/Documents/ironhack/AI_Engineering/Work/Week4/lab-natural-language-processing-1/data/kg_train.csv", encoding='latin-1')
data.fillna("", inplace=True)  # Handle missing values

# Apply the clean_html function to clean the 'text' column
data['cleaned_text'] = data['text'].apply(clean_html)

# Check the unique values in the 'label' column
unique_labels = data['label'].unique()
print("Unique label values:", unique_labels)

# Adjust the following lines based on your label values
if 'ham' in unique_labels and 'spam' in unique_labels:
    ham_messages = data[data['label'] == 'ham']['cleaned_text']
    spam_messages = data[data['label'] == 'spam']['cleaned_text']
else:
    # Example adjustment: if 0 is for ham and 1 is for spam
    ham_messages = data[data['label'] == 0]['cleaned_text']
    spam_messages = data[data['label'] == 1]['cleaned_text']

# If no valid messages are found, raise an error
if ham_messages.empty or spam_messages.empty:
    raise ValueError("Ensure the dataset contains correctly labeled messages (e.g., 'ham' and 'spam' or 0 and 1)")

# Step 1: Initialize CountVectorizer for Bag of Words
vectorizer_ham = CountVectorizer()
vectorizer_spam = CountVectorizer()

# Step 2: Fit and transform the ham and spam messages separately
ham_bow = vectorizer_ham.fit_transform(ham_messages)
spam_bow = vectorizer_spam.fit_transform(spam_messages)

# Step 3: Sum word frequencies for both ham and spam
ham_word_counts = np.asarray(ham_bow.sum(axis=0)).flatten()
spam_word_counts = np.asarray(spam_bow.sum(axis=0)).flatten()

# Step 4: Create DataFrames to store word counts for ham and spam separately
ham_word_freq = pd.DataFrame({'word': vectorizer_ham.get_feature_names_out(), 'count': ham_word_counts})
spam_word_freq = pd.DataFrame({'word': vectorizer_spam.get_feature_names_out(), 'count': spam_word_counts})

# Step 5: Get top 10 words for ham and spam
top_10_ham = ham_word_freq.sort_values(by='count', ascending=False).head(10)
top_10_spam = spam_word_freq.sort_values(by='count', ascending=False).head(10)

# Display the top 10 words for ham and spam
print("Top 10 words in ham messages:")
print(top_10_ham)

print("\nTop 10 words in spam messages:")
print(top_10_spam)





C:\Users\mktmi\AppData\Local\Temp\ipykernel_14144\2523896845.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')
C:\Users\mktmi\AppData\Local\Temp\ipykernel_14144\2523896845.py:16: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')


Unique label values: [1 0]
Top 10 words in ham messages:
       word  count
28927   the  13765
30086    to   8369
1433    and   6694
19932    of   6507
14083    in   4892
28799  that   3065
15068    is   2874
10870   for   2747
20348    on   2578
33108  with   1899

Top 10 words in spam messages:
       word  count
37061   the  38709
38258    to  31169
27174    of  27601
2387    and  21392
19198    in  17473
42238   you  16285
37826  this  14051
42457  your  11666
25555    my  11635
15390   for  11618


## Extra features

In [30]:
import re
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the clean_html function
def clean_html(html_content):
    # Remove inline JavaScript/CSS
    html_content = re.sub(r'<(script|style).*?>.*?</\1>', '', html_content, flags=re.DOTALL)
    # Remove HTML comments
    html_content = re.sub(r'<!--.*?-->', '', html_content, flags=re.DOTALL)
    # Remove remaining HTML tags
    soup = BeautifulSoup(html_content, 'html.parser')
    cleaned_text = soup.get_text()
    # Remove special characters (keeping only letters and spaces)
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', cleaned_text)
    # Remove numbers
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    # Remove single characters and substitute multiple spaces with a single space
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    # Convert to lowercase
    return cleaned_text.lower().strip()

# Load your dataset
data = pd.read_csv("C:/Users/mktmi/Documents/ironhack/AI_Engineering/Work/Week4/lab-natural-language-processing-1/data/kg_train.csv", encoding='latin-1')

# Apply the clean_html function to clean the 'text' column
data['preprocessed_text'] = data['text'].apply(clean_html)

# Split the dataset into training and validation sets
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

# Lists for money symbols and suspicious words
money_symbol_list = "|".join([r"\beuro\b", r"\bdollar\b", r"\bpound\b", r"€", r"\$"])
suspicious_words = "|".join([r"\bfree\b", r"\bcheap\b", r"\bsex\b", r"\bmoney\b", r"\baccount\b", r"\bbank\b", 
                             r"\bfund\b", r"\btransfer\b", r"\btransaction\b", r"\bwin\b", r"\bdeposit\b", r"\bpassword\b"])

# Add indicators to the training set
data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_symbol_list, regex=True) * 1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words, regex=True) * 1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

# Add indicators to the validation set
data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_symbol_list, regex=True) * 1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words, regex=True) * 1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

# Show the updated training data
data_train.head()


C:\Users\mktmi\AppData\Local\Temp\ipykernel_14144\3049394148.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')
C:\Users\mktmi\AppData\Local\Temp\ipykernel_14144\3049394148.py:13: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')


,text,label,preprocessed_text,money_mark,suspicious_words,text_len
3821,back -- we need to talkCall only my berry.,0,back we need to talkcall only my berry,0,0,38
2407,What is mullen's first name?,0,what is mullens first name,0,0,26
1173,Greetings from Dubai=2CThis letter must come t...,1,greetings from dubaicthis letter must come to ...,0,1,3726
4231,Sullivan Jacob J <SullivanJJ@state.gov>Friday ...,0,sullivan jacob friday december amiranthe eu me...,0,0,575
1666,Fyi,0,fyi,0,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample text corpus
corpus = [
    "I love NLP. NLP is amazing.",
    "I love machine learning.",
    "NLP and machine learning are great."
]

# Step 1: Initialize CountVectorizer
vectorizer = CountVectorizer()

# Step 2: Fit the CountVectorizer to the text data
X = vectorizer.fit_transform(corpus)

# Step 3: Show the vocabulary (words and their index)
print("Vocabulary:", vectorizer.vocabulary_)

# Step 4: Convert the document-term matrix to an array (for better visualization)
print("BoW representation (array):\n", X.toarray())

# Step 5: Display the feature names (words in the vocabulary)
print("Feature names (words):", vectorizer.get_feature_names_out())


Vocabulary: {'love': 6, 'nlp': 8, 'is': 4, 'amazing': 0, 'machine': 7, 'learning': 5, 'and': 1, 'are': 2, 'great': 3}
BoW representation (array):
 [[1 0 0 0 1 0 1 0 2]
 [0 0 0 0 0 1 1 1 0]
 [0 1 1 1 0 1 0 1 1]]
Feature names (words): ['amazing' 'and' 'are' 'great' 'is' 'learning' 'love' 'machine' 'nlp']


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load your dataset (replace with the actual path to your dataset)
data = pd.read_csv("C:/Users/mktmi/Documents/ironhack/AI_Engineering/Work/Week4/lab-natural-language-processing-1/data/kg_train.csv", encoding='latin-1')

# Assuming the dataset has a 'preprocessed_text' column containing the cleaned text
corpus = data_train['preprocessed_text'].tolist()

# Step 1: Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Step 2: Fit and transform the dataset (vectorize the corpus)
X = vectorizer.fit_transform(corpus)

# Step 3: Print the shape of the vectorized dataset (documents x terms)
print("Shape of the vectorized dataset:", X.shape)

# Step 4: Display feature names (words in the vocabulary)
print("Feature names (words):", vectorizer.get_feature_names_out())

# (Optional) Step 5: Display the TF-IDF matrix in array form
print("TF-IDF matrix:\n", X.toarray())


Shape of the vectorized dataset: (4771, 61758)
Feature names (words): ['aa' 'aaadecfbbefdc' 'aabidjan' ... 'zusichere' 'zwei' 'zyedc']
TF-IDF matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## And the Train a Classifier?

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Load your dataset
data = pd.read_csv("C:/Users/mktmi/Documents/ironhack/AI_Engineering/Work/Week4/lab-natural-language-processing-1/data/kg_train.csv", encoding='latin-1')

# Step 2: Assuming the dataset has a 'preprocessed_text' column and a 'label' column (ham/spam)
corpus = data_train['preprocessed_text'].tolist()
labels = data_train['label'].tolist()

# Step 3: Initialize the TfidfVectorizer and vectorize the dataset
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Step 5: Initialize the classifier (Logistic Regression)
classifier = LogisticRegression()

# Step 6: Train the classifier on the training data
classifier.fit(X_train, y_train)

# Step 7: Predict the labels for the test data
y_pred = classifier.predict(X_test)

# Step 8: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Step 9: Print detailed classification report
print("Classification Report:\n", classification_report(y_test, y_pred))



Accuracy: 0.9591623036649215
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.96       538
           1       0.99      0.91      0.95       417

    accuracy                           0.96       955
   macro avg       0.97      0.95      0.96       955
weighted avg       0.96      0.96      0.96       955



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset from the given Kaggle link (assuming CSV format)
data = pd.read_csv("path_to_spam_dataset.csv", encoding='latin-1')

# Step 1: Preprocess the dataset
# Assuming the dataset has a 'text' column and 'label' column ('spam' or 'ham')
# Clean the text (example cleaning function)
def clean_text(text):
    # Lowercasing and removing punctuation, numbers, and extra spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning
data['cleaned_text'] = data['text'].apply(clean_text)

# Step 2: Feature Representation
# Try different feature representations
# Example: You can toggle between CountVectorizer and TfidfVectorizer

# Option 1: Bag of Words (CountVectorizer)
# vectorizer = CountVectorizer()

# Option 2: TF-IDF
vectorizer = TfidfVectorizer()

# Transform the text data into features
X = vectorizer.fit_transform(data['cleaned_text'])

# Step 3: Train-Test Split
# Assuming 'label' column is binary (0 for ham, 1 for spam)
X_train, X_test, y_train, y_test = train_test_split(X, data['label'], test_size=0.2, random_state=42)

# Step 4: Train the Classifier (Multinomial Naive Bayes)
classifier = MultinomialNB()

# Train the model
classifier.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = classifier.predict(X_test)

# Step 6: Print Accuracy and Classification Report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

# Optional: Display feature names (if needed)
print("Feature names:\n", vectorizer.get_feature_names_out())


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_spam_dataset.csv'